# Mission to Mars

In [2]:
from  bs4 import  BeautifulSoup
import pandas as pd
import requests
import re

In [5]:
text  =  requests.get('https://mars.nasa.gov/news').text
data = re.split("<a href='/news/", text)[1]
title = re.split("'", re.split("</div>\n<img alt='", data)[1])[0]
paragraph = re.split(" \n</div>\n<div class='overlay_arrow'>", re.split("<div class='rollover_description_inner'>\n", data)[1])[0]
if "&quot;" in paragraph:
    paragraph = re.sub("&quot;", r'"', paragraph)
img = re.split("-", re.split("_", re.split("' src=", re.split("/system/news_items/list_view_images/", data)[1])[0])[1])[0]
image_url = f'https://www.jpl.nasa.gov/spaceimages/images/largesize/{img}_hires.jpg'

weather = requests.get('https://twitter.com/marswxreport').text

mar_weather = 'Sol' + re.split('</p>\n</div>', weather.split('<p class="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text" lang="en" data-aria-label-part="0">Sol')[1])[0]

if "&quot;" in mar_weather:
    mar_weather = re.sub("&quot;", r'"', mar_weather)

In [6]:
# Get Mars Facts
facts = requests.get('https://space-facts.com/mars/').text

facts_dict = {}
names = ['Equatorial Diameter', 'Polar Diameter', 'Mass', 'Moons', 'Orbit Distance', 'Orbit Period', 'Surface Temperature', 'First Record', 'Recorded By']

for name in names:
    try:
        facts_dict[name] = re.split('<br />', facts.split(name+':</strong></td><td class="column-2">')[1])[0]
    except IndexError:
        facts_dict[name] = re.split('<br />', facts.split(name+': </strong></td><td class="column-2">')[1])[0]
    if '</td>' in facts_dict[name]:
        facts_dict[name] = re.split('</td>', facts_dict[name])[0]
    if '<a href=' in facts_dict[name]:
        facts_dict[name] = BeautifulSoup(facts_dict[name], 'html.parser')
        for a in facts_dict[name].findAll('a'):
            del a['href']
        facts_dict[name] = str(facts_dict[name])
    if '<a>' in facts_dict[name]:
        facts_dict[name] = facts_dict[name].replace('<a>', '')
    if '</a>' in facts_dict[name]:
        facts_dict[name] = facts_dict[name].replace('</a>', '')
    if '&amp;' in facts_dict[name]:
        facts_dict[name] = facts_dict[name].replace('&amp;', '&')

In [8]:
df = pd.DataFrame.from_dict(facts_dict, orient='index')
df.columns = ['Value']

In [11]:
spheres  = ['valles_marineris', 'cerberus', 'schiaparelli', 'syrtis_major']
titles = ['Valles Marineris Hemisphere', 'Cerberus Hemisphere', 'Schiaparelli Hemisphere', 'Syrtis Major Hemisphere']

In [10]:
df.to_html(border = '1', justify = 'center')

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: center;">\n      <th></th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass</th>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>Moons</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance</th>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature</th>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>First Record</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n</table>'

In [12]:
hemisphere_image_urls = []
for i in range(len(spheres)):
    dic = {}
    dic['title'] = titles[i]
    dic['img_url'] = f'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/{spheres[i]}_enhanced.tif/full.jpg'
    hemisphere_image_urls.append(dic)

In [14]:
hemisphere_image_urls

[{'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere'}]